<a href="https://colab.research.google.com/github/DefinitelyNik/ner_finetune_pipeline/blob/main/ner_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install seqeval
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manyli

In [2]:
!unzip /content/drive/MyDrive/1000_samples_21Jun.zip -d data

Archive:  /content/drive/MyDrive/1000_samples_21Jun.zip
  inflating: data/jpg/0.jpg          
  inflating: data/jpg/1.jpg          
  inflating: data/jpg/10.jpg         
  inflating: data/jpg/100.jpg        
  inflating: data/jpg/101.jpg        
  inflating: data/jpg/102.jpg        
  inflating: data/jpg/103.jpg        
  inflating: data/jpg/104.jpg        
  inflating: data/jpg/105.jpg        
  inflating: data/jpg/106.jpg        
  inflating: data/jpg/107.jpg        
  inflating: data/jpg/108.jpg        
  inflating: data/jpg/109.jpg        
  inflating: data/jpg/11.jpg         
  inflating: data/jpg/110.jpg        
  inflating: data/jpg/111.jpg        
  inflating: data/jpg/112.jpg        
  inflating: data/jpg/113.jpg        
  inflating: data/jpg/114.jpg        
  inflating: data/jpg/115.jpg        
  inflating: data/jpg/116.jpg        
  inflating: data/jpg/117.jpg        
  inflating: data/jpg/118.jpg        
  inflating: data/jpg/119.jpg        
  inflating: data/jpg/12.jpg    

In [3]:
!unzip /content/drive/MyDrive/scrapped_letters.zip -d validation

Archive:  /content/drive/MyDrive/scrapped_letters.zip
  inflating: validation/images/0f7904c2-13.jpg  
  inflating: validation/images/1a12fe10-6.jpg  
  inflating: validation/images/1aa7138a-19.jpg  
  inflating: validation/images/298123a4-1.jpg  
  inflating: validation/images/35a3f749-2.jpg  
  inflating: validation/images/39b8c57c-15.jpg  
  inflating: validation/images/549faf50-4.jpg  
  inflating: validation/images/67088c48-22.jpg  
  inflating: validation/images/6daf7751-17.jpg  
  inflating: validation/images/6db47cd2-9.jpg  
  inflating: validation/images/6facf8da-8.jpg  
  inflating: validation/images/7cc17ca6-16.jpg  
  inflating: validation/images/804fe001-0.jpg  
  inflating: validation/images/84fdf028-5.jpg  
  inflating: validation/images/8789aaef-24.jpg  
  inflating: validation/images/88a32c2e-3.jpg  
  inflating: validation/images/a76fc5b5-14.jpg  
  inflating: validation/images/aab0305b-21.jpg  
  inflating: validation/images/b1102976-11.jpg  
  inflating: validation/

##Dataset generation

In [4]:
from os import listdir
from os.path import isfile, join
import json
from datasets import Dataset,DatasetDict,Features,ClassLabel,features

In [5]:
dataset_json = []
path_json ='/content/data/json/'
path_img = '/content/data/jpg/'


onlyfiles = [f for f in listdir(path_json) if isfile(join(path_json, f))]
for f in onlyfiles:
  json_file_path = path_json + f
  with open(json_file_path, 'r') as j:
     contents = json.load(j)
     dataset_json.append(contents)
for item in dataset_json:
  item['boxes'] = [value[0]+value[1] for value in item['boxes']]

In [6]:
dataset_json[0].keys()

dict_keys(['filename', 'boxes', 'words', 'ner_tags', 'clusters'])

In [7]:
#class_names = ['O', 'I-SND', 'B-SND', 'I-PHN', 'B-PHN', 'I-ORG', 'B-ORG', 'I-LOC', 'B-LOC', 'I-JT', 'B-JT', 'I-EXR', 'B-EXR', 'B-MN', 'B-MAIL', 'B-KPP', 'B-INN', 'I-DS', 'B-DS', 'I-ADR', 'B-ADR', 'I-MN']

In [8]:
class_names = ['O', 'B-SND', 'I-SND', 'B-PHN', 'I-PHN', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-JT', 'I-JT', 'B-EXR', 'I-EXR', 'B-MN', 'I-MN', 'B-DS', 'I-DS', 'B-ADR', 'I-ADR', 'B-MAIL', 'I-MAIL', 'B-KPP', 'I-KPP', 'B-INN', 'I-INN']

In [9]:
val_json = []
path_json ='/content/validation/scrapped_letters.json'
path_img = '/content/validation/jpg/'

with open(path_json, 'r') as j:
  contents = json.load(j)
  val_json.append(contents)

In [10]:
val_json[0][0].keys()

dict_keys(['ocr', 'id', 'bbox', 'label', 'transcription', 'annotator', 'annotation_id', 'created_at', 'updated_at', 'lead_time'])

##Preprocess data functions

In [11]:
import numpy as np
import collections
import math

In [12]:
def unnormalize_box(x0, y0, width, height, original_width, original_height):
         x0 = (x0 / 100) * original_width
         y0 = (y0 / 100) * original_height
         x1 = x0 + (width/100 * original_width)
         y1 = y0 + (height/100 * original_height)
         return [x0, y0, x1, y1]

In [13]:
def line_overlap(line_params, box): # line_params - Tuple[y0, y1], box - List[x0, y0, x1, y1]
    intersection = max(0, min(line_params[1], box[3]) - max(line_params[0], box[1]))
    return intersection / (line_params[1] - line_params[0])

def sort_by_lines(boxes):
    lines_dict = collections.defaultdict(list)
    for box_i, box in enumerate(boxes):
        if len(lines_dict.keys()) == 0:
            lines_dict[(box[1], box[3])].append((box_i, box))
        else:
            belongs_line = [line_overlap(k, box) > 0.5 for k in lines_dict]
            if any(belongs_line):
                line_p = list(lines_dict.keys())[belongs_line.index(True)]
                lines_dict[line_p].append((box_i, box))
            else:
                lines_dict[(box[1], box[3])].append((box_i, box))
    bbox_by_line = sorted([(line[0], bbox) for line, bbox in lines_dict.items()], key=lambda x: x[0])
    nested_boxes = [sorted(b, key=lambda x: x[1][0]) for _, b in bbox_by_line]
    bboxes = [box[1] for line in nested_boxes for box in line]
    bboxes_i = [box[0] for line in nested_boxes for box in line]
    return np.array(bboxes), bboxes_i

In [14]:
def order_boxes(boxes):
    def contour_dist(box1, box2):
        def axiswise_dist(s1, s2):
            """
            s = (left coord, right coord)
            """
            left = min(s1, s2, key=lambda x: x[0])
            right = max(s1, s2, key=lambda x: x[0])

            return max(0, right[0] - left[1])

        delta_x = axiswise_dist((box1[0], box1[2]), (box2[0], box2[2]))
        delta_y = axiswise_dist((box1[1], box1[3]), (box2[1], box2[3]))

        if delta_x == delta_y == 0:
            return 0
        if min(delta_x, delta_y) == 0 and max(delta_x, delta_y) > 0:
            return max(delta_x, delta_y)
        return math.sqrt(delta_x * delta_x + delta_y * delta_y)

    # y_sorted_boxes = np.array(sorted(boxes.copy(), key=lambda x: x[1]))
    # y_sorted_boxes_indices = sorted(range(len(boxes)), key=lambda x: boxes[x][1])
    y_sorted_boxes, y_sorted_boxes_indices = sort_by_lines(boxes.copy())
    epsilon = np.mean(boxes[:, 3] - boxes[:, 1])

    cluster_labels = np.full((len(y_sorted_boxes, )), fill_value=-1)

    cluster_labels[0] = 0

    for i in range(len(y_sorted_boxes)):
        if cluster_labels[i] == -1:
            cluster_labels[i] = max(cluster_labels) + 1
        for j in range(len(y_sorted_boxes)):
            if contour_dist(y_sorted_boxes[i], y_sorted_boxes[j]) < epsilon:
                min_l = min(cluster_labels[i], cluster_labels[j])
                max_l = max(cluster_labels[i], cluster_labels[j])
                if min_l == -1:
                    cluster_labels[i] = max_l
                    cluster_labels[j] = max_l
                    continue
                cluster_labels[cluster_labels == max_l] = min_l

    cluster_labels = np.array(cluster_labels)

    # for i in range(max(cluster_labels)):
    #     if i not in cluster_labels:
    #         cluster_labels[cluster_labels[cluster_labels > i].min()] = i

    sorted_boxes = list()
    for c in np.unique(cluster_labels):
        cluster_boxes = y_sorted_boxes[cluster_labels == c]
        cluster_indices = np.where(cluster_labels == c)[0]

        group = [y_sorted_boxes_indices[cluster_indices[0]]]
        for i in cluster_indices[1:]:
            if abs(y_sorted_boxes[y_sorted_boxes_indices[i]][1] - y_sorted_boxes[y_sorted_boxes_indices[i - 1]][1]) < (epsilon*0.5):
                group.append(y_sorted_boxes_indices[i])
            else:
                sorted_boxes.extend(sorted(group, key=lambda x: y_sorted_boxes[y_sorted_boxes_indices[x]][0]))
                group = [y_sorted_boxes_indices[i]]
        sorted_boxes.extend(sorted(group, key=lambda x: y_sorted_boxes[y_sorted_boxes_indices[x]][0]))

    return sorted_boxes

In [15]:
id2label = {v: k for v, k in enumerate(class_names)}
label2id = {k: v for v, k in enumerate(class_names)}

In [16]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [17]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [18]:
#letter crop function

def new_letter_segmentation(example):
  result = []
  letter_length = len(example['input_ids'])
  letter_samples = math.ceil(letter_length/512)
  window_length = math.ceil(letter_length/letter_samples)
  dynamic_window_length = 0

  for i in range(letter_samples):
    data = {}
    start = 0 + dynamic_window_length
    end = dynamic_window_length + window_length
    data['input_ids'] = example['input_ids'][start:end]
    data['token_type_ids'] = example['token_type_ids'][start:end]
    data['attention_mask'] = example['attention_mask'][start:end]
    data['word_ids'] = example.word_ids(0)[start:end]
    result.append(data)
    dynamic_window_length += window_length

  return result

In [19]:
def convert_ner_tags_to_id(ner_tags):
  return [label2id[ner_tag] for ner_tag in ner_tags]

In [20]:
def convert_id_to_ner_tags(ids):
  return [id2label[id] for id in ids]

In [21]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["words"], truncation=False, is_split_into_words=True
    )
    tokenized_inputs_array = new_letter_segmentation(tokenized_inputs)
    for item in tokenized_inputs_array:
        all_labels = convert_ner_tags_to_id(examples['ner_tags'])
        word_ids = item['word_ids']
        new_labels = align_labels_with_tokens(all_labels, word_ids)
        item["labels"] = new_labels
    return tokenized_inputs_array

##Words, labels and boxes sort

In [22]:
val_dataset = []

for item in val_json[0]:
  val_data = {}

  unnormalize_boxes = []
  for bbox in item['bbox']:
    x0 = bbox['x']
    y0 = bbox['y']
    width = bbox['width']
    height = bbox['height']
    original_width = bbox['original_width']
    original_height = bbox['original_height']
    unnormalize_boxes.append(unnormalize_box(x0, y0, width, height, original_width, original_height))

  np_unnormalize_boxes = np.array(unnormalize_boxes)
  sorted_indices = order_boxes(np_unnormalize_boxes)
  sorted_boxes = np_unnormalize_boxes[sorted_indices]

  labels = []
  for label in item['label']:
    labels.append(label['labels'][0])

  np_labels = np.array(labels)
  sorted_labels = np_labels[sorted_indices]

  words = item['transcription']
  stripped_words = []

  for word in words:
    stripped_words.append(word.strip())

  np_words = np.array(stripped_words)
  sorted_words = np_words[sorted_indices]

  val_data['filename'] = item['id']
  val_data['boxes'] = sorted_boxes
  val_data['words'] = sorted_words
  val_data['ner_tags'] = sorted_labels

  val_dataset.append(val_data)

print(len(val_dataset))
print(val_dataset[0].keys())

25
dict_keys(['filename', 'boxes', 'words', 'ner_tags'])


In [23]:
train_dataset = []

for item in dataset_json:
  train_data = {}

  train_np_boxes = np.array(item['boxes'])
  sorted_ids = order_boxes(train_np_boxes)
  train_sorted_boxes = train_np_boxes[sorted_ids]

  train_words = item['words']
  train_stripped_words = []

  for word in train_words:
    train_stripped_words.append(word.strip())

  train_np_words = np.array(train_stripped_words)
  train_sorted_words = train_np_words[sorted_ids]

  train_np_labels = np.array(item['ner_tags'])
  train_sorted_labels = train_np_labels[sorted_ids]

  train_data['filename'] = item['filename']
  train_data['boxes'] = train_sorted_boxes
  train_data['words'] = train_sorted_words
  train_data['ner_tags'] = train_sorted_labels

  train_dataset.append(train_data)

print(len(train_dataset))
print(train_dataset[0].keys())

1000
dict_keys(['filename', 'boxes', 'words', 'ner_tags'])


In [24]:
train = Dataset.from_list(train_dataset).shuffle()
val = Dataset.from_list(val_dataset)

In [25]:
dataset = DatasetDict({
    'train': train,
    'validation': val})
dataset

DatasetDict({
    train: Dataset({
        features: ['filename', 'boxes', 'words', 'ner_tags'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['filename', 'boxes', 'words', 'ner_tags'],
        num_rows: 25
    })
})

In [26]:
train = []

for train_item in dataset['train']:
  data = tokenize_and_align_labels(train_item)
  for item in data:
    train.append(item)

print(len(train))
print(train[0].keys())

Token indices sequence length is longer than the specified maximum sequence length for this model (612 > 512). Running this sequence through the model will result in indexing errors


1077
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'])


In [27]:
validation = []

for valid_item in dataset['validation']:
  data = tokenize_and_align_labels(valid_item)
  for item in data:
    validation.append(item)

print(len(validation))
print(validation[0].keys())

52
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'])


In [28]:
final_train = Dataset.from_list(train).shuffle()
final_val = Dataset.from_list(validation)

In [29]:
final_train = final_train.select_columns(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
final_val = final_val.select_columns(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [30]:
final_dataset = DatasetDict({
    'train': final_train,
    'validation': final_val})
final_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1077
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 52
    })
})

##Metrics

In [31]:
import evaluate

metric = evaluate.load("seqeval")

##Train, save and upload

In [32]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [33]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    final_dataset["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    final_dataset["validation"], collate_fn=data_collator, batch_size=8
)

In [34]:
model = AutoModelForTokenClassification.from_pretrained(
    "Babelscape/wikineural-multilingual-ner",
    ignore_mismatched_sizes=True,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Babelscape/wikineural-multilingual-ner and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([25]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([25, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [36]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [37]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [38]:
from huggingface_hub import Repository, get_full_repo_name, notebook_login

notebook_login()

In [40]:
model_name = "wikineural_finetuned_test"
repo_name = get_full_repo_name(model_name)
repo_name

'Heyzz/wikineural_finetuned_test'

In [41]:
import os

output_dir = "new_finetuned_model"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Heyzz/wikineural_finetuned_test into local empty directory.


Download file model.safetensors:   0%|          | 16.0k/676M [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/676M [00:00<?, ?B/s]

In [42]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[class_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [class_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [43]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))
counter = 0
step = 0

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        counter += 1

        if counter == 10:
            # Evaluation
            model.eval()
            step += 1
            for batch in eval_dataloader:
                with torch.no_grad():
                    outputs = model(**batch)

                predictions = outputs.logits.argmax(dim=-1)
                labels = batch["labels"]

                # Necessary to pad predictions and labels for being gathered
                predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
                labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

                predictions_gathered = accelerator.gather(predictions)
                labels_gathered = accelerator.gather(labels)

                true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
                metric.add_batch(predictions=true_predictions, references=true_labels)

            results = metric.compute()
            print(
                f"step: {step}:",
                {
                    key: results[f"overall_{key}"]
                    for key in ["precision", "recall", "f1", "accuracy"]
                },
            )
            counter = 0

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/405 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


step: 1: {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.6597355512902537}
step: 2: {'precision': 0.007125890736342043, 'recall': 0.007614213197969543, 'f1': 0.007361963190184049, 'accuracy': 0.5174877372574109}
step: 3: {'precision': 0.0166270783847981, 'recall': 0.029288702928870293, 'f1': 0.021212121212121213, 'accuracy': 0.6452868415440393}
step: 4: {'precision': 0.05938242280285035, 'recall': 0.040983606557377046, 'f1': 0.04849660523763336, 'accuracy': 0.6050330560887183}
step: 5: {'precision': 0.0831353919239905, 'recall': 0.026217228464419477, 'f1': 0.03986332574031891, 'accuracy': 0.4153870761356366}
step: 6: {'precision': 0.0855106888361045, 'recall': 0.026568265682656828, 'f1': 0.040540540540540536, 'accuracy': 0.4781936447003626}
step: 7: {'precision': 0.1377672209026128, 'recall': 0.03634085213032581, 'f1': 0.05751115518096182, 'accuracy': 0.40936233738537003}
step: 8: {'precision': 0.0997624703087886, 'recall': 0.04430379746835443, 'f1': 0.061358655953250546, 'a